# Energy Data Science Experiment


# Motivation

Being able to accurately forecast the volatile demand for power, especially across low-voltage distribution grids, is increasingly important to providing sufficent energy supply. Paradoxically, it's at the low-voltage networks that are the most volatile, and yet the most critical to account for as more distributed energy resources (e.g., solar, storage) make their way onto the grid. From the grid operator's point of view, not only are lower voltage service points (e.g., customers) the revenue life blood of the business, having increased visibility into demand at such lower levels of aggregation helps to provide 


In this notebook, I explore looking at a Kaggle data set containing about 5,000 utility service points with time series data from advanced metering infrastructure (AMI). The context of this experiment is to simulate the needs of a utility or grid operator and generate accurate short-term (up to 7 days ahead) forecast. I explore possible approaches that involve

- *Feature-based modeling approaches*
- *Customer Segmentation + Clustering*
- *Statistical methods, ML methods, and hybrid cross-learning approaches*

Performance is evaluated using time-series cross-validation using a rolling 7-day forecast horizon.

Finally, I show how this solution might be scaled and put into production.

Keywords: *time series classification*, *energy forecasting*, *cross-learning*, *MLOps*


In [2]:
# Import Dependencies

### Utilities ###

import os
import glob
#import boto3
import s3fs
from tqdm import tqdm

### Data Processing ###

import json
import numpy as np
import pandas as pd
from datetime import datetime

### PLOTTING ###

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)

import seaborn as sns
from plotnine import *
import plotnine as pl

### DATA SCIENCE ###

#import lightgbm as lgb
import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, confusion_matrix,classification_report
from sklearn.model_selection import GridSearchCV,KFold,RandomizedSearchCV
from scipy.stats import uniform, truncnorm, randint
import random
from functools import reduce


ModuleNotFoundError: No module named 'plotnine'

In [ ]:
# Set Project Working Directory
project_path = "/Users/jonfarland/Dropbox/08-Projects/showcase/"

#os.getcwd()
os.chdir(project_path)

#os.environ
#pd.set_option('display.max_columns', None)

data_path = f"{project_path}data"
data_path
 
# Read in Secure Credentials from Local File
#with open('cred.json') as f:
#  cred = json.load(f)

# Set AWS Credentials
#os.environ["AWS_ACCESS_KEY_ID"] = cred['aws'].get("AWS_ACCESS_KEY_ID")
#os.environ["AWS_SECRET_ACCESS_KEY"] = cred['aws'].get("AWS_SECRET_ACCESS_KEY")

# Set Twillio Credentials
#os.environ["TWILIO_SID"] = cred['twilio'].get("cred1")
#os.environ["TWILIO_TOKEN"] = cred['twilio'].get("cred2")
#os.environ


In [ ]:
# Data 

#https://www.kaggle.com/jeanmidev/smart-meters-in-london

#hh = pd.read_csv(data_path + "halfhourly_dataset")
all_files = os.listdir("data/archive/hhblock_dataset/hhblock_dataset")

df_from_each_file = (pd.read_csv(data_path + "/archive/hhblock_dataset/hhblock_dataset/" + f, sep=',') for f in all_files)
df_merged   = pd.concat(df_from_each_file, ignore_index=True)


In [ ]:
df_merged.head()

In [ ]:
print(df_merged.shape)
print(df_merged.describe)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(df_merged["day"], df_merged["hh_0"])


In [ ]:
#df = df_merged[df_merged['LCLid'] == 'MAC000027']
#df.head()

## Tranpose from wide to long

df = pd.wide_to_long(df_merged, stubnames = "hh_", i = ['day', 'LCLid'], j = "hh").sort_values(["day", "hh"]).reset_index()

#df.columns.values
#df.shape

#ax.plot(df0["hh_"])
fig, ax = plt.subplots()
ax.plot(df["hh"], df["hh_"])


In [ ]:
# import time lookup
lookup = pd.read_csv(data_path + "/time-lookup.csv", sep = ",")
lookup.head()

In [ ]:
df = df.sort_values(["LCLid", "day", "hh"])
df = df.join(lookup.set_index("hh"), how = "left", on = "hh")
df.head()

In [ ]:
df["timestamp"] = pd.to_datetime(df['day'] + " " + df['time'])

In [ ]:
print(df.shape)
print(df.head())

In [ ]:
df = df.drop(["day", "hh", "time"], axis = 1)
df.head()

In [ ]:
df.columns = ["id", "kwh", "timestamp"]
df.head()

In [ ]:
df_wide = df.pivot(index = "timestamp", columns = 'id', values = 'kwh')

df_wide.head()

In [ ]:
df_wide = df_wide.to_numpy()
df_wide[np.isnan(df_wide)] = 0

In [ ]:
print(df_wide.shape)
print(np.transpose(df_wide).shape)

In [ ]:
series = np.transpose(df_wide)
X_train, y_train = series[:4500, :39695], series[:4500, -1]
X_valid, y_valid = series[4500:5000, :39695], series[4500:5000, -1]
X_test, y_test = series[5000:, :39695], series[5000:, -1]

In [ ]:
X_train, y_train = X_train[~np.isnan(X_train)], y_train[~np.isnan(y_train)]
X_valid, y_valid = X_valid[~np.isnan(X_valid)], y_valid[~np.isnan(y_valid)]
X_test, y_test = X_test[~np.isnan(X_test)], y_test[~np.isnan(y_test)]

In [ ]:
y_pred = X_valid[:, -1]
np.mean(keras.losses.mean_squared_error(y_valid[~np.isnan(y_valid)], y_pred[~np.isnan(y_pred)]))

In [ ]:
import tensorflow as tf
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[39695, 1]),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=100,
                    validation_data=(X_valid, y_valid))

In [ ]:
model.evaluate(X_valid, y_valid)

In [ ]:
def plot_learning_curves(loss, val_loss):
    plt.plot(np.arange(len(loss)) + 0.5, loss, "b.-", label="Training loss")
    plt.plot(np.arange(len(val_loss)) + 1, val_loss, "r.-", label="Validation loss")
    plt.gca().xaxis.set_major_locator(mpl.ticker.MaxNLocator(integer=True))
    plt.axis([1, 100, 0, 30])
    plt.legend(fontsize=14)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.grid(True)

plot_learning_curves(history.history["loss"], history.history["val_loss"])
plt.show()

In [ ]:
y_pred = model.predict(X_valid)
plot_series(X_valid[0, :, 0], y_valid[0, 0], y_pred[0, 0])
plt.show()

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

optimizer = keras.optimizers.Adam(learning_rate=0.005)
model.compile(loss="mse", optimizer=optimizer)
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid))

In [ ]:
# Unique Number of Sensor meters
ids = pd.DataFrame(df_merged['LCLid'].unique()).sample(n = 1000)
ids.columns = ['id']

df = df.loc[df['LCLid'].isin(ids["id"])]
df.head()


In [ ]:
#df = df_merged[df_merged['LCLid'] == 'MAC000027']
#df.head()

df0 = pd.wide_to_long(df_merged, stubnames = "hh_", i = ['day', 'LCLid'], j = "hh").sort_values(["day", "hh"])
df = df0.reset_index()

#df.columns.values
df.shape

#ax.plot(df0["hh_"])
#fig, ax = plt.subplots()
#ax.scatter(df["hh"], df["hh_"])

In [ ]:
df_wide = df.pivot(index = "timestamp", columns = 'id', values = 'usage')

In [ ]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  #   wave 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + wave 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)   # + noise
    return series[..., np.newaxis].astype(np.float32)

In [ ]:
np.random.seed(42)

n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]




In [ ]:
##############################
### User Defined Functions ###
##############################

# Define a function to recursively flatten JSON object
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

# Define a function to process each JSON record
def process_row(x):
    input_json = json.loads(x)
    flat_json = flatten_json(input_json)
    flat_df = pd.DataFrame.from_dict(flat_json, orient = "index").T
    return flat_df

# Define a function process an entire file
def process_file(filename, destination = "data/processed/"):
	df = pd.read_excel(filename)

	# Extract Participant ID
	pid = df.participant_id.iloc[0]

	# Split data based on data category
	ops_data = df[df['data_category'] == 'Operational']
	conf_data = df[df['data_category'] == 'Configuration']

	# Use List Comprehension to apply function to each row
	list_output = [process_row(x) for x in tqdm(ops_data['hvac'])]

	# Concatenate list output to produce data frame
	df_output = pd.concat(list_output, ignore_index = True)

	# Concatenate Processed JSON with IDs
	output = pd.concat((ops_data.drop(['hvac'], axis = 1).reset_index(), df_output), axis=1)

	# Establish Output File Name
	output_file_name = destination + pid + '-ops.csv'

	# Writeout operational data
	output.to_csv(output_file_name)

	# Use List Comprehension to apply function to each row
	list_output = [process_row(x) for x in tqdm(conf_data['hvac'])]

	# Concatenate list output to produce data frame
	df_output = pd.concat(list_output, ignore_index = True)

	# Concatenate Processed JSON with IDs
	output = pd.concat((conf_data.drop(['hvac'], axis = 1).reset_index(), df_output), axis=1)

	# Establish Output File Name
	output_file_name = destination + pid + '-config.csv'

	# Writeout configuration data data
	output.to_csv(output_file_name)


# list files in current directory
directory = os.getcwd()

# iterate over files in current directory and process them

for filename in os.listdir(directory):
    if filename.endswith(".xlsx"):
        try:
        	print("Processing File: " + filename)
        	process_file(filename, destination = "/Users/jonfarland/Documents/projects/APC/data/processed/")
        except:
        	print("Error Processing File: " + filename)
    else:
        continue
